## Boston Housing Price Prediction with Tensorflow & Keras
In this notebook we illustrate a Regression problem, where we predict price of a house in the Boston area given several predictors. We will use Tensorflow 2.0 & Keras to prepare the model.

In [5]:
import sys, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

print(f"Using Tensorflow version {tf.__version__}")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

Using Tensorflow version 2.12.0


**NOTE:** The Boston Housing dataset has been deprecated from `sklearn.datasets` package as it had ethical issues. Please see [this article](https://towardsdatascience.com/things-you-didnt-know-about-the-boston-housing-dataset-2e87a6f960e8) for an excellent explanation why. Also, the good news is that we have a corrected dataset, which is available at [the corrected boston dataset link](http://lib.stat.cmu.edu/datasets/boston_corrected.txt)

In [15]:
# downloading data from the corrected dataset link below
# NOTE: we need to skip the first 13 rows before we get to the data, which is
def get_data(force_download=False):
    if not os.path.exists("./data/boston_housing.csv") or force_download:
        df = pd.read_csv(
            "http://lib.stat.cmu.edu/datasets/boston_corrected.txt", sep="\t", skiprows=8, encoding="cp1252"
        )
    else:
        df = pd.read_csv("./data/boston_housing.csv", index_col=0)
    return df

In [18]:
df = get_data()
# df.head()

df = df.drop(["OBS."], axis=1)
df.head()

,TOWN,TOWN#,TRACT,LON,LAT,MEDV,CMEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,Nahant,0,2011,-70.955,42.2550,24.0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,Swampscott,1,2021,-70.950,42.2875,21.6,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,Swampscott,1,2022,-70.936,42.2830,34.7,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,Marblehead,2,2031,-70.928,42.2930,33.4,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,Marblehead,2,2032,-70.922,42.2980,36.2,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33
